In [2]:
import pandas as pd
import numpy as np
import json
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv
import google.generativeai as genai
import time

load_dotenv('.env')

fireworks_key = os.environ.get("FIREWORKS_KEY")
mistral_model="accounts/fireworks/models/mistral-7b-instruct-v0p2"
llama_model = "accounts/fireworks/models/llama-v3-8b-instruct"
phi_model = "accounts/fireworks/models/phi-3-mini-128k-instruct"
fireworks_client = Fireworks(api_key=fireworks_key)

TEMP = 0
MAX_TOKENS = 2500

### Generating Advice using small LLMs with Zero-Shot and Few-Shot Approach

In [3]:
def generate_prompt(input, relationship_type, conversational_context, person, few_shot=True):
    """
    Generate a prompt for a conversational model based on input parameters.

    Parameters:
    input (str): The user's input conversation text.
    relationship_type (str): Type of relationship (e.g., 'Romantic Relationship', 'Friendship').
    conversational_context (str): The conversational context (e.g., 'I feel like I may have acted wrong').
    person (str): The person's role in the conversation ('culprit' or 'victim').
    few_shot (bool): Flag to indicate if few-shot learning context should be included.

    Returns:
    list: A list of prompt dictionaries for the chat model.
    """
    
    # Mapping relationship types to file naming convention
    relationship_dict = {
        "Romantic Relationship": "relationship", 
        "Friendship": "friendship", 
        "Workplace (Colleagues)": "workplace-no-hierarchy", 
        "Workplace (Manager)": "workplace-hierarchy"
    }

    # Mapping conversational contexts to file naming convention
    conversational_contexts_dict = {
        "I feel like I may have acted wrong": "Advice-for-culprit",
        "I feel I might have been mistreated": "Advice for victim",
        "I don't know what to say next": "Next-reply"
    }

    # Load the few-shot prompt from the appropriate file
    with open(f"../../../assistant/few_shot_prompts/{relationship_dict.get(relationship_type)}-{conversational_contexts_dict.get(conversational_context)}.json", "r") as file:
        prompt = json.load(file)

    # Descriptions for the relationship context in the conversation
    relationship_types = {
        "Romantic Relationship": f"This is a conversation between me ({person}) and my romantic partner", 
        "Friendship": f"This is a conversation between me ({person}) and my friends", 
        "Workplace (Colleagues)": f"This is a conversation between me ({person}) and my colleagues", 
        "Workplace (Manager)": f"This is a conversation between me ({person}) and my manager"
    }

    # Descriptions for the conversational context
    conversational_contexts = {
        "I feel like I may have acted wrong": "I feel I might have acted wrong in the conversation. Did I make any mistakes? What could I do better?",
        "I feel I might have been mistreated": "The conversation leaves me uneasy, but I do not know why. What went wrong in this interaction?",
        "I don't know what to say next": "I don't like the way this interaction went. What should I say next? Why would that be the right reaction?"
    }

    # Construct the user's input string with relationship and conversational context
    user_string = f"""{relationship_types.get(relationship_type)} 

                        {input} 
                        
                        {conversational_contexts.get(conversational_context)}"""
    
    # Append to existing prompt if few_shot is True, otherwise create a new prompt
    if few_shot:
        prompt.append({"role": "user", "content": user_string})
    else:
        prompt = [{"role": "user", "content": user_string}]

    return prompt

def get_fireworks_completion(messages, fireworks_model):
    """
    Generate a completion using the Fireworks API based on the provided messages.

    Parameters:
    messages (list): List of message dictionaries for the chat model.
    fireworks_model (str): The model to be used for generating the completion.

    Returns:
    dict: The chat completion response from the Fireworks API.
    """
    # Request a chat completion from the Fireworks API
    chat_completion = fireworks_client.chat.completions.create(
        model=fireworks_model,
        messages=messages,
        max_tokens=MAX_TOKENS,
        temperature=TEMP,   
    )
    return chat_completion

In [20]:
few_shot = pd.read_csv("not_few_shot_chats_rel.csv")
relationship_few_shot = few_shot[few_shot["context"] == "Relationship"].copy()

In [21]:
results = {}  # Initialize an empty dictionary to store results

# Iterate over each row in the relationship_few_shot DataFrame
for index, row in relationship_few_shot.iterrows():
    results[index] = {}  # Initialize a nested dictionary for each row's results
    results[index]["chat_text"] = row["chat_text"]  # Store the chat text
    results[index]["Victim"] = row["Victim"]  # Store the victim's role
    results[index]["culprit"] = row["culprit"]  # Store the culprit's role
    
    # Generate a few-shot prompt for the given context
    few_shot_prompt = generate_prompt(row["chat_text"], "Romantic Relationship", "I feel I might have been mistreated", row["Victim"])
    
    # Get few-shot completions from different models
    few_shot_mistral = get_fireworks_completion(few_shot_prompt, mistral_model)
    few_shot_llama = get_fireworks_completion(few_shot_prompt, llama_model)
    few_shot_phi = get_fireworks_completion(few_shot_prompt, phi_model)
    
    # Generate a zero-shot prompt for the given context
    zero_shot_prompt = generate_prompt(row["chat_text"], "Romantic Relationship", "I feel I might have been mistreated", row["Victim"], few_shot=False)
    
    # Get zero-shot completions from different models
    zero_shot_mistral = get_fireworks_completion(zero_shot_prompt, mistral_model)
    zero_shot_llama = get_fireworks_completion(zero_shot_prompt, llama_model)
    zero_shot_phi = get_fireworks_completion(zero_shot_prompt, phi_model)
    
    # Store the completion texts in the results dictionary
    results[index]["few_shot_mistral"] = few_shot_mistral.choices[0].message.content
    results[index]["zero_shot_mistral"] = zero_shot_mistral.choices[0].message.content
    results[index]["few_shot_llama"] = few_shot_llama.choices[0].message.content
    results[index]["zero_shot_llama"] = zero_shot_llama.choices[0].message.content
    results[index]["few_shot_phi"] = few_shot_phi.choices[0].message.content
    results[index]["zero_shot_phi"] = zero_shot_phi.choices[0].message.content

# Convert the results dictionary to a DataFrame
df_responses = pd.DataFrame.from_dict(results, orient='index')

# Save the DataFrame to a CSV file
df_responses.to_csv("open_source_advice_for_victim.csv")


### Evaluation of the quality of generated advice using rater model

In [9]:
# Reading in generated open-source advice
open_source_resp = pd.read_csv("open_source_advice_for_victim.csv")
# Configure Google genai
genai.configure(api_key=os.environ["API_KEY"])

In [6]:
# Configure model and generation_config

generation_config = {
  "temperature": 0,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

model = genai.GenerativeModel('gemini-1.5-flash',
                              generation_config=generation_config,)

In [10]:
def get_response(response_column, row):

    """
    Generate a prompt to evaluate the quality of advice given the interaction context and relationship between speakers,
    and get the response from a model.

    Parameters:
    response_column (str): The column name containing the advice to be evaluated.
    row (Series): A row from the DataFrame containing chat text and other relevant information.

    Returns:
    str: The response text from the model.
    """

    # Initialize the prompt with instructions and context
    this = "Input: Now, judge if this next advice given is of high quality given the interaction context and relationship between the speakers. \n"
    this += "The following is a Whatsapp conversation between a romantic couple. \n"
    # Add the chat text to the prompt
    this += f"""```{row["chat_text"]}```\n"""
    this += f"""The following is advice for {row["Victim"]} who felt uneasy after the conversation\n"""
    # Add the advice text to the prompt
    this += f"""```{row[response_column]}```\n"""
    # Add the rating instructions to the prompt
    this += """On a scale from 1 to 5, how good would you judge this last advice? 
                Provide an explanation for your rating first, then give your rating using this JSON schema: 
                    {"explanation": str,
                    "rating": int}"""
    
    # Define the initial and retry prompt content
    prompt_content = ["input: First have a look at the examples below. They were judged with respect to their quality given the interaction context and relationship between the speakers. Only take these as examples for what to look out for when rating advice later on. I do not want you to rate these.",
            "output: Ok, let me see the examples.",
            "input: The following is a Email conversation between colleagues. \n```John: Hey everyone, I wanted to touch base about the project deadline. As per our last meeting, we agreed to have everything wrapped up by Friday. Let's make sure we're all on track!\n\nSarah: Hi John, I thought we decided on next Wednesday as the deadline during the meeting. I even have it written down in my notes. Did I misunderstand something?\n\nJohn: Sarah, I'm pretty sure we all agreed on Friday. I distinctly remember discussing it and everyone nodding in agreement. Is your memory playing tricks on you?\n\nMike: I'm with Sarah on this one. I also have next Wednesday written down as the deadline we agreed upon.\n\nJohn: Guys, I don't know what to tell you. I clearly remember Friday being the consensus. I even mentioned it to our supervisor after the meeting, and she seemed on board with it. Maybe you both misheard me?```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: It's a good advice, especially clearing any further misunderstanding and avoiding questioning someone's memory that would lead to a strainous relationship. \n\n\n Rating: 4",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: This is good advice the romantic relationship is important but maintaining relationships with friends is also important. Clear boundaries need to be set and  no one should be manipulated to do something they feel uncomfortable with\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice is resonates with what I was thinking while reading the chat, there is some lack of reciprocity in the conversation and one should not feel bad about that especially under the circumstances of an immature relationship. The advice considers all aspects of the relationship; commutation, feelings and truth.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for Anna who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: What I liked about the advice is that it acknowledged the way in which the plans that David has/or had are dismissed. It's the selfish approach he recieved and the way in which his plans are found unimportant.\n\nI think the whole advice is likable and correct \n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```Subject: Project Proposal Feedback\n\nEmail 1:\nFrom: John Smith\nTo: Team\nSubject: Project Proposal Feedback\n\nHi Team,\n\nI hope everyone is doing well. I just finished reviewing the project proposal that Amy prepared. While I appreciate the effort she put into it, I have noticed several issues that I believe we need to address as a team.\n\nFirstly, the objective of the project was not clearly defined. I assumed that we all agreed on the fact that customer satisfaction is our main goal. But it seems like Amy has focused on other aspects, like cost-cutting and efficiency, which are not our primary objectives at this stage.\n\nAlso, there was a lack of research and data to back up the strategies suggested, which I believe is a crucial part of any proposal. \n\nI don't mean to criticize, but I think it's essential for us to have these things in mind for future proposals. \n\nBest,\nJohn\n\nEmail 2:\nFrom: Amy Johnson\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHello All,\n\nThank you for your feedback, John. I will make sure to focus more on customer satisfaction and include more data in future proposals. \n\nBest,\nAmy\n\nEmail 3:\nFrom: John Smith\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHi Amy,\n\nGood to hear that. I believe this will improve the quality of your work and contribute to the success of our upcoming projects. \n\nTeam, let's take this as an example for all of us to learn from. We should always aim for quality work and ensure we do thorough research before submitting our proposals.\n\nBest,\nJohn```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice clearly lays out the various parts of the email that could be improved. By using numbered points and detailed comments, John can further learn how to give better feedback.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice is resonates with what I was thinking while reading the chat, there is some lack of reciprocity in the conversation and one should not feel bad about that especially under the circumstances of an immature relationship. The advice considers all aspects of the relationship; commutation, feelings and truth.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day?\n\nB: It was good, did a lot at work today. How about you?\n\nA: Work was fine. You know I was just thinking, you're so cute when you try hard at work.\n\nB: Haha, I always try hard at work.\n\nA: Yeah, I know. But it's like you're trying to prove something. It's adorable.\n\nB: Well, I just want to do my best.\n\nA: Of course, and it's cute. You're like a little kid with a big dream. \n\nB: Haha, I guess so.\n\nA: But it's good that you're ambitious. Not everyone can be a natural at their job, right?\n\nB: Yeah, you're right. \n```\nThe following is Advice for A who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I love the advice as it will help the interaction be better and maybe open room for many more interactions.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Person A: Hey guys, just wanted to share my good news. I got promoted at work!\n\nPerson B: Oh wow, congrats! You must be over the moon.\n\nPerson C: That's awesome, A! You've been working hard.\n\nPerson D: Congrats, A. I'm surprised they didn't promote you sooner considering how long you've been there.\n\nPerson A: Thanks, guys. I'm really excited.\n\nPerson B: You should be! It's a big step up.\n\nPerson C: Definitely. We should celebrate.\n\nPerson D: Yes, definitely. After all, it's not everyday you manage to impress your bosses. \n\nPerson A: Thanks, D. That means a lot. \n\nPerson B: How about drinks on Saturday?\n\nPerson C: Sounds great. Congrats again, A.\n\nPerson D: Yes, congrats. You always had a knack for getting noticed when it counts.```\nThe following is Advice for Person D who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice is impressive and makes one understand the subtle impressions that a simple conversation could have/make on both the recipient and the sender. It teaches me that in a congratulatory conversation, one should be careful of making ambiguous statements that might be interpreted in many ways by other people and that one should be mindful of how the conversation flows between people.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for A who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I love that the advise doesn't only cover the situation at hand but it gives pointer/ precautions for future issues that may arise. It also gives advice on how to move from the issue at hand by correcting manipulative behaviour \n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```John: Hey guys, I need some help moving this weekend. Anyone available to lend a hand?\n\nSarah: I wish I could, but I have a family event on Saturday. Sorry, John!\n\nMike: I might be able to help for a couple of hours on Sunday afternoon. What time were you thinking?\n\nJohn: I was hoping to get it all done on Saturday, but Sunday afternoon could work too. I'll need help with some heavy lifting and packing. Pizza and beers on me!\n\nMike: Alright, I can probably make Sunday work. Just let me know the details later this week.\n\nJohn: Thanks, Mike! I really appreciate it. By the way, didn't you say I could borrow your truck for the move? That would be a huge help!\n\nMike: Oh, um, I don't remember saying that. I might need the truck this weekend.\n\nJohn: Really? I could've sworn you mentioned it when I helped you paint your living room last month. No worries if you can't, but I was counting on it.\n\nMike: Ah, I guess I did say that. Sure, you can borrow the truck. Just fill up the tank when you're done.\n\nJohn: You're a lifesaver, Mike! Thanks a bunch. See you Sunday!```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: It’s really goof advise, but I dislike they compensation part because they are friends and beer n pizza is enough if thats how they usually chill with friends… i like that the advice highlights that John was wrong for guilt tripping Mike into borrowing him his truck \n Rating: 3",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The boss was unfair as the colleagues were taking what he was saying as a teachable moment for them and a wayu for them to gain more knowledge they would take with the to future work and that would have been possible if the boss was open to teaching them or fixing their errors but it seemed as though he was undermining them and choosing to do it himself as he had more experience which surely rubbed off the wrong way to the colleagues who did not mind learning\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I love that the advice gives clear points that needed to be addressed and these points are addressed perfectly aa even there I do feel like I agree with A I'm still able to agree with A at some points while still setting my boundaries.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```John: Hey guys, I need some help moving this weekend. Anyone available to lend a hand?\n\nSarah: I wish I could, but I have a family event on Saturday. Sorry, John!\n\nMike: I might be able to help for a couple of hours on Sunday afternoon. What time were you thinking?\n\nJohn: I was hoping to get it all done on Saturday, but Sunday afternoon could work too. I'll need help with some heavy lifting and packing. Pizza and beers on me!\n\nMike: Alright, I can probably make Sunday work. Just let me know the details later this week.\n\nJohn: Thanks, Mike! I really appreciate it. By the way, didn't you say I could borrow your truck for the move? That would be a huge help!\n\nMike: Oh, um, I don't remember saying that. I might need the truck this weekend.\n\nJohn: Really? I could've sworn you mentioned it when I helped you paint your living room last month. No worries if you can't, but I was counting on it.\n\nMike: Ah, I guess I did say that. Sure, you can borrow the truck. Just fill up the tank when you're done.\n\nJohn: You're a lifesaver, Mike! Thanks a bunch. See you Sunday!```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I think this advice was perfect. Moving is a very big task, so it definitely does need to be planned well in advance, which is why the \"Give more notice\" point is perfect. Considering this, it was unfair to give a specific day over the weekend when John wanted help, as his friends likely have their own plans, hence \"Be flexible\" makes sense. The biggest issue was how John asked for the truck, where he assumed that his friend would remember agreeing to this. This could have maybe been unpacked more in the advice, however, overall the advice was good.\n Rating: 4",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The provided advice considers both of the partners feelings, behaviours and attitudes towards each other.\nIt is not judgemental as it highlights \"mutual respect, trust and compromise\" as these are key to a healthy relationship.\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```From: Sarah\nTo: Michael, Tom\nSubject: Urgent: Project Deadline\n\nHey Michael and Tom,\n\nI hope you're both doing well. I wanted to touch base regarding the project deadline coming up next week. I've been working tirelessly to ensure everything is on track, but I'm concerned that we might not meet the deadline if we don't put in some extra effort.\n\nMichael, I know you've been dealing with some personal issues lately, but I really need you to step up and contribute more to the project. We're all counting on you.\n\nTom, you've been doing a great job so far, but I think we could use your expertise to help us finalize the presentation. Can you please take a look at it and provide some feedback?\n\nI'd hate to let the team down, so let's all work together to make this project a success. I'm willing to put in the extra hours, and I hope you both are too.\n\nBest,\nSarah\n\nFrom: Michael\nTo: Sarah, Tom\nSubject: Re: Urgent: Project Deadline\n\nHi Sarah,\n\nThanks for reaching out. I apologize for not being as involved as I should be. You're right, I have been dealing with some personal matters, but I understand the importance of this project. I'll do my best to contribute more and help us meet the deadline.\n\nTom, let me know if you need any assistance with the presentation.\n\nBest regards,\nMichael\n\nFrom: Tom\nTo: Sarah, Michael\nSubject: Re: Urgent: Project Deadline\n\nHey Sarah and Michael,\n\nNo worries, I'm happy to review the presentation and provide feedback. I'll make sure to have it done by tomorrow EOD.\n\nMichael, let me know if you need any help with your tasks as well. We're all in this together!\n\nCheers,\nTom```\nThe following is Advice for Sarah who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: Everyone has taken on board everyone else's point of view, and have moved on in an effective manner.\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```Subject: Project Proposal Feedback\n\nEmail 1:\nFrom: John Smith\nTo: Team\nSubject: Project Proposal Feedback\n\nHi Team,\n\nI hope everyone is doing well. I just finished reviewing the project proposal that Amy prepared. While I appreciate the effort she put into it, I have noticed several issues that I believe we need to address as a team.\n\nFirstly, the objective of the project was not clearly defined. I assumed that we all agreed on the fact that customer satisfaction is our main goal. But it seems like Amy has focused on other aspects, like cost-cutting and efficiency, which are not our primary objectives at this stage.\n\nAlso, there was a lack of research and data to back up the strategies suggested, which I believe is a crucial part of any proposal. \n\nI don't mean to criticize, but I think it's essential for us to have these things in mind for future proposals. \n\nBest,\nJohn\n\nEmail 2:\nFrom: Amy Johnson\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHello All,\n\nThank you for your feedback, John. I will make sure to focus more on customer satisfaction and include more data in future proposals. \n\nBest,\nAmy\n\nEmail 3:\nFrom: John Smith\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHi Amy,\n\nGood to hear that. I believe this will improve the quality of your work and contribute to the success of our upcoming projects. \n\nTeam, let's take this as an example for all of us to learn from. We should always aim for quality work and ensure we do thorough research before submitting our proposals.\n\nBest,\nJohn```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: 1.true as this may create conflict in the team\n2.its important to speak to everyone in a calm manner\n3.true\n4.this twill help with communication going forward\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```From: Sarah\nTo: Michael, Tom\nSubject: Urgent: Project Deadline\n\nHey Michael and Tom,\n\nI hope you're both doing well. I wanted to touch base regarding the project deadline coming up next week. I've been working tirelessly to ensure everything is on track, but I'm concerned that we might not meet the deadline if we don't put in some extra effort.\n\nMichael, I know you've been dealing with some personal issues lately, but I really need you to step up and contribute more to the project. We're all counting on you.\n\nTom, you've been doing a great job so far, but I think we could use your expertise to help us finalize the presentation. Can you please take a look at it and provide some feedback?\n\nI'd hate to let the team down, so let's all work together to make this project a success. I'm willing to put in the extra hours, and I hope you both are too.\n\nBest,\nSarah\n\nFrom: Michael\nTo: Sarah, Tom\nSubject: Re: Urgent: Project Deadline\n\nHi Sarah,\n\nThanks for reaching out. I apologize for not being as involved as I should be. You're right, I have been dealing with some personal matters, but I understand the importance of this project. I'll do my best to contribute more and help us meet the deadline.\n\nTom, let me know if you need any assistance with the presentation.\n\nBest regards,\nMichael\n\nFrom: Tom\nTo: Sarah, Michael\nSubject: Re: Urgent: Project Deadline\n\nHey Sarah and Michael,\n\nNo worries, I'm happy to review the presentation and provide feedback. I'll make sure to have it done by tomorrow EOD.\n\nMichael, let me know if you need any help with your tasks as well. We're all in this together!\n\nCheers,\nTom```\nThe following is Advice for Michael who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I think that regardless of having issues we should be able to meet deadlines, I believe that Michael can use the work as as an excuse or escape from his personal life and actually contribute his part\n Rating: 3",
            "input: The following is a Whatsapp conversation between friends. \n```Person A: Hey guys, just wanted to share my good news. I got promoted at work!\n\nPerson B: Oh wow, congrats! You must be over the moon.\n\nPerson C: That's awesome, A! You've been working hard.\n\nPerson D: Congrats, A. I'm surprised they didn't promote you sooner considering how long you've been there.\n\nPerson A: Thanks, guys. I'm really excited.\n\nPerson B: You should be! It's a big step up.\n\nPerson C: Definitely. We should celebrate.\n\nPerson D: Yes, definitely. After all, it's not everyday you manage to impress your bosses. \n\nPerson A: Thanks, D. That means a lot. \n\nPerson B: How about drinks on Saturday?\n\nPerson C: Sounds great. Congrats again, A.\n\nPerson D: Yes, congrats. You always had a knack for getting noticed when it counts.```\nThe following is Advice for Person A who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: What I like is that the advicer explained very well to what might make this person A to feel like he/she didn't deserve to be happy,it's because his friends didn't focus on him being promoted but the focus on why should this person be promoted.\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: Emphasizing the team's hard work and investment in the proposal underlines the commitment and seriousness with which the team approaches their work. This can help the boss appreciate the effort and potentially reconsider their initial assessment.\n Rating: 4",
            "input: The following is a Email conversation between employees and their boss. \n```From: Michael Johnson ",
            "output: Explanation: The 1,2,3 steps are easy to get and quite spot on. They provide great advice that can easily be applied. Maybe giving text examples after each piece of advice can be even more beneficial.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day?\n\nB: It was good, did a lot at work today. How about you?\n\nA: Work was fine. You know I was just thinking, you're so cute when you try hard at work.\n\nB: Haha, I always try hard at work.\n\nA: Yeah, I know. But it's like you're trying to prove something. It's adorable.\n\nB: Well, I just want to do my best.\n\nA: Of course, and it's cute. You're like a little kid with a big dream. \n\nB: Haha, I guess so.\n\nA: But it's good that you're ambitious. Not everyone can be a natural at their job, right?\n\nB: Yeah, you're right. \n```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The feedback has useful comments on how to deal with this situation and let your partner know where you stand with this sort of behaviour. The example reply given is also good as it gives an example of how to act in the future if such a comment is made to you.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: 1 star because everyone should be a priority over friends when you in a relationship \n\n Rating: 1",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day?\n\nB: It was good, did a lot at work today. How about you?\n\nA: Work was fine. You know I was just thinking, you're so cute when you try hard at work.\n\nB: Haha, I always try hard at work.\n\nA: Yeah, I know. But it's like you're trying to prove something. It's adorable.\n\nB: Well, I just want to do my best.\n\nA: Of course, and it's cute. You're like a little kid with a big dream. \n\nB: Haha, I guess so.\n\nA: But it's good that you're ambitious. Not everyone can be a natural at their job, right?\n\nB: Yeah, you're right. \n```\nThe following is Advice for A who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I love the advice as it will help the interaction be better and maybe open room for many more interactions.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day?\n\nB: It was good, did a lot at work today. How about you?\n\nA: Work was fine. You know I was just thinking, you're so cute when you try hard at work.\n\nB: Haha, I always try hard at work.\n\nA: Yeah, I know. But it's like you're trying to prove something. It's adorable.\n\nB: Well, I just want to do my best.\n\nA: Of course, and it's cute. You're like a little kid with a big dream. \n\nB: Haha, I guess so.\n\nA: But it's good that you're ambitious. Not everyone can be a natural at their job, right?\n\nB: Yeah, you're right. \n```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I think the advice the person has given is very accurate, but from my point of view it depends on the relationship between the two people, let me explain:\nA sensitive person with a tendency to overthink things may interpret person A's answers as devaluing, and that he or she is indeed not grateful for the work done.\nOn the other hand, if the relationship between them is clear, and if both protagonists know how the other reacts, I think person B might take it well. A couple's relationship isn't just a conversation in writing, but in everyday life.\n\nIn any case, I think that the best way to resolve this conflict (if there is one) is to talk about it face-to-face.\n Rating: 4",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for A who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation:  The feedback was constructive and informed me of what I did wrong and how to improve. It was broken down in a clear way and it had a positive tone.\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: Similar to first scenario, I think this is perfect advice. It provides a list of potential issues, which can help person B verbalize it to their partner in an open conversation.\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Project Deadline Concerns\n\nSender: Boss\nDear Team,\n\nI hope this message finds you well.\n\nI've been reviewing our project timelines and I'm a bit concerned that we're not moving at the pace we should be. Remember, our competitors are also working on similar projects and if we don't pick up the pace, they might beat us to the punch.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nBoss,\n\nThanks for your message. We're doing our best to keep on schedule. I believe we're making good progress, considering the setbacks we've had. \n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nColleague 1,\n\nI understand we've had setbacks. However, we need to understand the gravity of the situation. If we don't meet our deadlines, there could be serious consequences for the future of our team. Job security is no longer a guarantee in our industry. I'm sure none of us want to be in a position where we are looking for work.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nBoss,\n\nWe understand the importance of the project and we're putting in extra hours to make sure we meet the deadline. We appreciate your concern and assure you we're doing everything we can.\n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nColleague 2,\n\nI'm glad to hear that. I just want all of us to understand what's at stake here.\nBest,\nBoss```\nThe following is Advice for Boss who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I strongly agree with the advice because no person wants to work in an environment wherein their job is always at threat.\nI believe a happy employee is a very productive employee, of course targets must be met however there should me a nicer way of conveying the message.\n\nI think the advice is correct and encourages a more positive working environment \n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```Subject: Project Proposal Feedback\n\nEmail 1:\nFrom: John Smith\nTo: Team\nSubject: Project Proposal Feedback\n\nHi Team,\n\nI hope everyone is doing well. I just finished reviewing the project proposal that Amy prepared. While I appreciate the effort she put into it, I have noticed several issues that I believe we need to address as a team.\n\nFirstly, the objective of the project was not clearly defined. I assumed that we all agreed on the fact that customer satisfaction is our main goal. But it seems like Amy has focused on other aspects, like cost-cutting and efficiency, which are not our primary objectives at this stage.\n\nAlso, there was a lack of research and data to back up the strategies suggested, which I believe is a crucial part of any proposal. \n\nI don't mean to criticize, but I think it's essential for us to have these things in mind for future proposals. \n\nBest,\nJohn\n\nEmail 2:\nFrom: Amy Johnson\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHello All,\n\nThank you for your feedback, John. I will make sure to focus more on customer satisfaction and include more data in future proposals. \n\nBest,\nAmy\n\nEmail 3:\nFrom: John Smith\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHi Amy,\n\nGood to hear that. I believe this will improve the quality of your work and contribute to the success of our upcoming projects. \n\nTeam, let's take this as an example for all of us to learn from. We should always aim for quality work and ensure we do thorough research before submitting our proposals.\n\nBest,\nJohn```\nThe following is Advice for Amy who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice affirmed my own thoughts and validated the feelings Amy would have. I think the advice did not get the lack of specifics part right as John's email was sort of specific.\n Rating: 4",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Project Deadline Concerns\n\nSender: Boss\nDear Team,\n\nI hope this message finds you well.\n\nI've been reviewing our project timelines and I'm a bit concerned that we're not moving at the pace we should be. Remember, our competitors are also working on similar projects and if we don't pick up the pace, they might beat us to the punch.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nBoss,\n\nThanks for your message. We're doing our best to keep on schedule. I believe we're making good progress, considering the setbacks we've had. \n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nColleague 1,\n\nI understand we've had setbacks. However, we need to understand the gravity of the situation. If we don't meet our deadlines, there could be serious consequences for the future of our team. Job security is no longer a guarantee in our industry. I'm sure none of us want to be in a position where we are looking for work.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nBoss,\n\nWe understand the importance of the project and we're putting in extra hours to make sure we meet the deadline. We appreciate your concern and assure you we're doing everything we can.\n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nColleague 2,\n\nI'm glad to hear that. I just want all of us to understand what's at stake here.\nBest,\nBoss```\nThe following is Advice for Boss who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: There are lots of ways to deal with a situation instead of just neglecting your other responsibilities, especially the ones that put food on the table. No matter how bad things are , they get worse if you don’t have money. Show up at your work and give it your best \n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for A who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I love that the advise doesn't only cover the situation at hand but it gives pointer/ precautions for future issues that may arise. It also gives advice on how to move from the issue at hand by correcting manipulative behaviour \n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I think the boss/employee dynamic should be considered here. Although the advice given was valid, it also assumed quite a bit.This type of advice is helpful for the boss to receive and not colleague 1. Advice to colleague 1 could be to ask for a meeting or how to best navigate the situation further.\n Rating: 3",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Project Deadline Concerns\n\nSender: Boss\nDear Team,\n\nI hope this message finds you well.\n\nI've been reviewing our project timelines and I'm a bit concerned that we're not moving at the pace we should be. Remember, our competitors are also working on similar projects and if we don't pick up the pace, they might beat us to the punch.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nBoss,\n\nThanks for your message. We're doing our best to keep on schedule. I believe we're making good progress, considering the setbacks we've had. \n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nColleague 1,\n\nI understand we've had setbacks. However, we need to understand the gravity of the situation. If we don't meet our deadlines, there could be serious consequences for the future of our team. Job security is no longer a guarantee in our industry. I'm sure none of us want to be in a position where we are looking for work.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nBoss,\n\nWe understand the importance of the project and we're putting in extra hours to make sure we meet the deadline. We appreciate your concern and assure you we're doing everything we can.\n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nColleague 2,\n\nI'm glad to hear that. I just want all of us to understand what's at stake here.\nBest,\nBoss```\nThe following is Advice for Boss who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I strongly agree with the advice because no person wants to work in an environment wherein their job is always at threat.\nI believe a happy employee is a very productive employee, of course targets must be met however there should me a nicer way of conveying the message.\n\nI think the advice is correct and encourages a more positive working environment \n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice was the right one as A was wrong and did not consider B at all and used manipulation tactics to get their way in a way it is very umhealthy and is unfair to the other partner but it seems as though A might have done this before seeing as to how she's comfortable threatening with possibilities of ending the relationship which also has negative effects on B\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```John: Hey guys, I need some help moving this weekend. Anyone available to lend a hand?\n\nSarah: I wish I could, but I have a family event on Saturday. Sorry, John!\n\nMike: I might be able to help for a couple of hours on Sunday afternoon. What time were you thinking?\n\nJohn: I was hoping to get it all done on Saturday, but Sunday afternoon could work too. I'll need help with some heavy lifting and packing. Pizza and beers on me!\n\nMike: Alright, I can probably make Sunday work. Just let me know the details later this week.\n\nJohn: Thanks, Mike! I really appreciate it. By the way, didn't you say I could borrow your truck for the move? That would be a huge help!\n\nMike: Oh, um, I don't remember saying that. I might need the truck this weekend.\n\nJohn: Really? I could've sworn you mentioned it when I helped you paint your living room last month. No worries if you can't, but I was counting on it.\n\nMike: Ah, I guess I did say that. Sure, you can borrow the truck. Just fill up the tank when you're done.\n\nJohn: You're a lifesaver, Mike! Thanks a bunch. See you Sunday!```\nThe following is Advice for Mike who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: What i liked about the advice was that it showed the needs of a healthy relationship and why it should be addressed but also we should not forget that people do make mistakes and it was not intentional. Again lastly i liked that it gave me a clear picture of whats happening and also touched on the fact that i'm being reminded of the past favour\n Rating: 3",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The provided advice considers both of the partners feelings, behaviours and attitudes towards each other.\nIt is not judgemental as it highlights \"mutual respect, trust and compromise\" as these are key to a healthy relationship.\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I liked that the advice shows the boss how he could be wrong and how to better respond.\n Rating: 4",
            "input: The following is a Email conversation between colleagues. \n```From: Sarah\nTo: Michael, Tom\nSubject: Urgent: Project Deadline\n\nHey Michael and Tom,\n\nI hope you're both doing well. I wanted to touch base regarding the project deadline coming up next week. I've been working tirelessly to ensure everything is on track, but I'm concerned that we might not meet the deadline if we don't put in some extra effort.\n\nMichael, I know you've been dealing with some personal issues lately, but I really need you to step up and contribute more to the project. We're all counting on you.\n\nTom, you've been doing a great job so far, but I think we could use your expertise to help us finalize the presentation. Can you please take a look at it and provide some feedback?\n\nI'd hate to let the team down, so let's all work together to make this project a success. I'm willing to put in the extra hours, and I hope you both are too.\n\nBest,\nSarah\n\nFrom: Michael\nTo: Sarah, Tom\nSubject: Re: Urgent: Project Deadline\n\nHi Sarah,\n\nThanks for reaching out. I apologize for not being as involved as I should be. You're right, I have been dealing with some personal matters, but I understand the importance of this project. I'll do my best to contribute more and help us meet the deadline.\n\nTom, let me know if you need any assistance with the presentation.\n\nBest regards,\nMichael\n\nFrom: Tom\nTo: Sarah, Michael\nSubject: Re: Urgent: Project Deadline\n\nHey Sarah and Michael,\n\nNo worries, I'm happy to review the presentation and provide feedback. I'll make sure to have it done by tomorrow EOD.\n\nMichael, let me know if you need any help with your tasks as well. We're all in this together!\n\nCheers,\nTom```\nThe following is Advice for Sarah who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I gave a star rating of 5 because the well being of employees is taken into consideration and their hard work does not go unnoticed that's why the company depends on these individuals for a better progress both the leader and team are having common ground \n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for Anna who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The feedback clearly pointed out where the response might have been passive-aggressive. This is crucial because it helped me see how Anna's words could be interpreted negatively, even if that wasn’t her intention. This kind of insight is valuable for personal growth and improving communication skills.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for David who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I gave the advice I 5 star rating as it accurately describes the situation and explains why David would be feeling this way. It takes in consideration Anna's comment and tone as well as the additional comments made by John and Lily.\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```From: Michael Johnson ",
            "output: Explanation: The 1,2,3 steps are easy to get and quite spot on. They provide great advice that can easily be applied. Maybe giving text examples after each piece of advice can be even more beneficial.\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```John: Hey everyone, I wanted to touch base about the project deadline. As per our last meeting, we agreed to have everything wrapped up by Friday. Let's make sure we're all on track!\n\nSarah: Hi John, I thought we decided on next Wednesday as the deadline during the meeting. I even have it written down in my notes. Did I misunderstand something?\n\nJohn: Sarah, I'm pretty sure we all agreed on Friday. I distinctly remember discussing it and everyone nodding in agreement. Is your memory playing tricks on you?\n\nMike: I'm with Sarah on this one. I also have next Wednesday written down as the deadline we agreed upon.\n\nJohn: Guys, I don't know what to tell you. I clearly remember Friday being the consensus. I even mentioned it to our supervisor after the meeting, and she seemed on board with it. Maybe you both misheard me?```\nThe following is Advice for Sarah who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I dislike the advice as it seems to be more of an analysis on the scenario rather than advice.\n Rating: 1",
            "input: The following is a Whatsapp conversation between friends. \n```John: Hey guys, I need some help moving this weekend. Anyone available to lend a hand?\n\nSarah: I wish I could, but I have a family event on Saturday. Sorry, John!\n\nMike: I might be able to help for a couple of hours on Sunday afternoon. What time were you thinking?\n\nJohn: I was hoping to get it all done on Saturday, but Sunday afternoon could work too. I'll need help with some heavy lifting and packing. Pizza and beers on me!\n\nMike: Alright, I can probably make Sunday work. Just let me know the details later this week.\n\nJohn: Thanks, Mike! I really appreciate it. By the way, didn't you say I could borrow your truck for the move? That would be a huge help!\n\nMike: Oh, um, I don't remember saying that. I might need the truck this weekend.\n\nJohn: Really? I could've sworn you mentioned it when I helped you paint your living room last month. No worries if you can't, but I was counting on it.\n\nMike: Ah, I guess I did say that. Sure, you can borrow the truck. Just fill up the tank when you're done.\n\nJohn: You're a lifesaver, Mike! Thanks a bunch. See you Sunday!```\nThe following is Advice for Mike who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: He shouldn’t have been rude to him but state that he will use it during the weekend \n Rating: 2",
            "input: The following is a Whatsapp conversation between friends. \n```Person A: Hey guys, just wanted to share my good news. I got promoted at work!\n\nPerson B: Oh wow, congrats! You must be over the moon.\n\nPerson C: That's awesome, A! You've been working hard.\n\nPerson D: Congrats, A. I'm surprised they didn't promote you sooner considering how long you've been there.\n\nPerson A: Thanks, guys. I'm really excited.\n\nPerson B: You should be! It's a big step up.\n\nPerson C: Definitely. We should celebrate.\n\nPerson D: Yes, definitely. After all, it's not everyday you manage to impress your bosses. \n\nPerson A: Thanks, D. That means a lot. \n\nPerson B: How about drinks on Saturday?\n\nPerson C: Sounds great. Congrats again, A.\n\nPerson D: Yes, congrats. You always had a knack for getting noticed when it counts.```\nThe following is Advice for Person D who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I think this is good advice and would likely be much better received by person .\n Rating: 5",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for B who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: Similar to first scenario, I think this is perfect advice. It provides a list of potential issues, which can help person B verbalize it to their partner in an open conversation.\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```John: Hey everyone, I wanted to touch base about the project deadline. As per our last meeting, we agreed to have everything wrapped up by Friday. Let's make sure we're all on track!\n\nSarah: Hi John, I thought we decided on next Wednesday as the deadline during the meeting. I even have it written down in my notes. Did I misunderstand something?\n\nJohn: Sarah, I'm pretty sure we all agreed on Friday. I distinctly remember discussing it and everyone nodding in agreement. Is your memory playing tricks on you?\n\nMike: I'm with Sarah on this one. I also have next Wednesday written down as the deadline we agreed upon.\n\nJohn: Guys, I don't know what to tell you. I clearly remember Friday being the consensus. I even mentioned it to our supervisor after the meeting, and she seemed on board with it. Maybe you both misheard me?```\nThe following is Advice for Sarah who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: 3 star because I understand that it's work but do not tell me that my mind is playing tricks \n Rating: 2",
            "input: The following is a Email conversation between colleagues. \n```From: Sarah\nTo: Michael, Tom\nSubject: Urgent: Project Deadline\n\nHey Michael and Tom,\n\nI hope you're both doing well. I wanted to touch base regarding the project deadline coming up next week. I've been working tirelessly to ensure everything is on track, but I'm concerned that we might not meet the deadline if we don't put in some extra effort.\n\nMichael, I know you've been dealing with some personal issues lately, but I really need you to step up and contribute more to the project. We're all counting on you.\n\nTom, you've been doing a great job so far, but I think we could use your expertise to help us finalize the presentation. Can you please take a look at it and provide some feedback?\n\nI'd hate to let the team down, so let's all work together to make this project a success. I'm willing to put in the extra hours, and I hope you both are too.\n\nBest,\nSarah\n\nFrom: Michael\nTo: Sarah, Tom\nSubject: Re: Urgent: Project Deadline\n\nHi Sarah,\n\nThanks for reaching out. I apologize for not being as involved as I should be. You're right, I have been dealing with some personal matters, but I understand the importance of this project. I'll do my best to contribute more and help us meet the deadline.\n\nTom, let me know if you need any assistance with the presentation.\n\nBest regards,\nMichael\n\nFrom: Tom\nTo: Sarah, Michael\nSubject: Re: Urgent: Project Deadline\n\nHey Sarah and Michael,\n\nNo worries, I'm happy to review the presentation and provide feedback. I'll make sure to have it done by tomorrow EOD.\n\nMichael, let me know if you need any help with your tasks as well. We're all in this together!\n\nCheers,\nTom```\nThe following is Advice for Michael who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: It deserves this rating because the adviser focused on how the interaction was professional and also acknowledged why Micheal might be feeling uneasy, it shows how Micheal shouldn't take things personally or overread situations.  \n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```John: Hey everyone, I wanted to touch base about the project deadline. As per our last meeting, we agreed to have everything wrapped up by Friday. Let's make sure we're all on track!\n\nSarah: Hi John, I thought we decided on next Wednesday as the deadline during the meeting. I even have it written down in my notes. Did I misunderstand something?\n\nJohn: Sarah, I'm pretty sure we all agreed on Friday. I distinctly remember discussing it and everyone nodding in agreement. Is your memory playing tricks on you?\n\nMike: I'm with Sarah on this one. I also have next Wednesday written down as the deadline we agreed upon.\n\nJohn: Guys, I don't know what to tell you. I clearly remember Friday being the consensus. I even mentioned it to our supervisor after the meeting, and she seemed on board with it. Maybe you both misheard me?```\nThe following is Advice for Sarah who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: 3, because the advice focuses more on siding with John than accommodating both parties. the first chat specified that two people were saying that they have Wednesday dotted down which means that John might be in the wrong so the solution here would be for the deadline to be rescheduled to another date and have all three parties write it down for proof. \n Rating: 3",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I liked that the advice shows the boss how he could be wrong and how to better respond.\n Rating: 4",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: Emphasizing the team's hard work and investment in the proposal underlines the commitment and seriousness with which the team approaches their work. This can help the boss appreciate the effort and potentially reconsider their initial assessment.\n Rating: 4",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Project Deadline Concerns\n\nSender: Boss\nDear Team,\n\nI hope this message finds you well.\n\nI've been reviewing our project timelines and I'm a bit concerned that we're not moving at the pace we should be. Remember, our competitors are also working on similar projects and if we don't pick up the pace, they might beat us to the punch.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nBoss,\n\nThanks for your message. We're doing our best to keep on schedule. I believe we're making good progress, considering the setbacks we've had. \n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nColleague 1,\n\nI understand we've had setbacks. However, we need to understand the gravity of the situation. If we don't meet our deadlines, there could be serious consequences for the future of our team. Job security is no longer a guarantee in our industry. I'm sure none of us want to be in a position where we are looking for work.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nBoss,\n\nWe understand the importance of the project and we're putting in extra hours to make sure we meet the deadline. We appreciate your concern and assure you we're doing everything we can.\n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nColleague 2,\n\nI'm glad to hear that. I just want all of us to understand what's at stake here.\nBest,\nBoss```\nThe following is Advice for Colleague 1 who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: This advice is very clear and detailed. I love how it points out specific areas of the conversation that one could have missed or not noticed. This advice brings to awareness a lot of things that the boss did which may be wrong and would need to be brought to the boss attention. I love this advice\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```Subject: Project Proposal Feedback\n\nEmail 1:\nFrom: John Smith\nTo: Team\nSubject: Project Proposal Feedback\n\nHi Team,\n\nI hope everyone is doing well. I just finished reviewing the project proposal that Amy prepared. While I appreciate the effort she put into it, I have noticed several issues that I believe we need to address as a team.\n\nFirstly, the objective of the project was not clearly defined. I assumed that we all agreed on the fact that customer satisfaction is our main goal. But it seems like Amy has focused on other aspects, like cost-cutting and efficiency, which are not our primary objectives at this stage.\n\nAlso, there was a lack of research and data to back up the strategies suggested, which I believe is a crucial part of any proposal. \n\nI don't mean to criticize, but I think it's essential for us to have these things in mind for future proposals. \n\nBest,\nJohn\n\nEmail 2:\nFrom: Amy Johnson\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHello All,\n\nThank you for your feedback, John. I will make sure to focus more on customer satisfaction and include more data in future proposals. \n\nBest,\nAmy\n\nEmail 3:\nFrom: John Smith\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHi Amy,\n\nGood to hear that. I believe this will improve the quality of your work and contribute to the success of our upcoming projects. \n\nTeam, let's take this as an example for all of us to learn from. We should always aim for quality work and ensure we do thorough research before submitting our proposals.\n\nBest,\nJohn```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: 1.true as this may create conflict in the team\n2.its important to speak to everyone in a calm manner\n3.true\n4.this twill help with communication going forward\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for Anna who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The feedback clearly pointed out where the response might have been passive-aggressive. This is crucial because it helped me see how Anna's words could be interpreted negatively, even if that wasn’t her intention. This kind of insight is valuable for personal growth and improving communication skills.\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Re: Project Proposal Feedback\n\nSender: Boss\nDear Team,\n\nI appreciate your efforts on the project proposal submitted last week. However, I think it needs a bit more refinement before we can present it to our clients. \n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nDear Boss,\n\nThank you for your feedback. Could you please specify which areas we need to refine?\n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nDear Colleague 1,\n\nI think the entire proposal could use a more professional touch. It's not something you'd understand at your level. I'll handle the refinement and presentation to the clients.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nDear Boss,\n\nWe are willing to make the necessary changes if you could guide us. \n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nDear Colleague 2,\n\nI appreciate your willingness to learn. But, as I said, this requires a level of finesse that comes with experience. I'll take care of the presentation, you should focus on your designated areas. \n\nBest,\nBoss```\nThe following is Advice for Colleague 1 who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The boss was unfair as the colleagues were taking what he was saying as a teachable moment for them and a wayu for them to gain more knowledge they would take with the to future work and that would have been possible if the boss was open to teaching them or fixing their errors but it seemed as though he was undermining them and choosing to do it himself as he had more experience which surely rubbed off the wrong way to the colleagues who did not mind learning\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```Subject: Project Proposal Feedback\n\nEmail 1:\nFrom: John Smith\nTo: Team\nSubject: Project Proposal Feedback\n\nHi Team,\n\nI hope everyone is doing well. I just finished reviewing the project proposal that Amy prepared. While I appreciate the effort she put into it, I have noticed several issues that I believe we need to address as a team.\n\nFirstly, the objective of the project was not clearly defined. I assumed that we all agreed on the fact that customer satisfaction is our main goal. But it seems like Amy has focused on other aspects, like cost-cutting and efficiency, which are not our primary objectives at this stage.\n\nAlso, there was a lack of research and data to back up the strategies suggested, which I believe is a crucial part of any proposal. \n\nI don't mean to criticize, but I think it's essential for us to have these things in mind for future proposals. \n\nBest,\nJohn\n\nEmail 2:\nFrom: Amy Johnson\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHello All,\n\nThank you for your feedback, John. I will make sure to focus more on customer satisfaction and include more data in future proposals. \n\nBest,\nAmy\n\nEmail 3:\nFrom: John Smith\nTo: Team\nSubject: RE: Project Proposal Feedback\n\nHi Amy,\n\nGood to hear that. I believe this will improve the quality of your work and contribute to the success of our upcoming projects. \n\nTeam, let's take this as an example for all of us to learn from. We should always aim for quality work and ensure we do thorough research before submitting our proposals.\n\nBest,\nJohn```\nThe following is Advice for Amy who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: What i liked about the advice was it touched on few key aspect of healthy working relationships. It clearly stated the wrongs john did and why he should be told\n Rating: 4",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, are you free tonight? I thought we could grab dinner and catch a movie.\n\nB: I'm sorry, I can't tonight. I promised my friends I'd join them for a game night. We've been planning this for a while.\n\nA: Oh, I see. So your friends are more important than spending time with me?\n\nB: No, that's not what I meant. It's just that I made these plans before you asked.\n\nA: Well, if you really cared about our relationship, you'd prioritize spending time with me.\n\nB: I do care about our relationship, but I also value my friendships. Can we plan something for tomorrow instead?\n\nA: I don't know. I'm starting to feel like I'm not a priority in your life.\n\nB: That's not true at all. You are a priority to me.\n\nA: Then prove it. Cancel your plans with your friends and come out with me tonight.\n\nB: I don't think it's fair to ask me to cancel last minute. My friends are counting on me.\n\nA: I'm not asking you, I'm telling you. If you don't come out with me tonight, I'm going to have to seriously reconsider our relationship.\n\nB: Okay, okay. I'll tell my friends I can't make it. Let's go out tonight.\n\nA: Great! I knew you'd make the right choice. See you later, love you!\n\nB: Love you too...```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: This is good advice the romantic relationship is important but maintaining relationships with friends is also important. Clear boundaries need to be set and  no one should be manipulated to do something they feel uncomfortable with\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```Subject: Project Deadline Concerns\n\nSender: Boss\nDear Team,\n\nI hope this message finds you well.\n\nI've been reviewing our project timelines and I'm a bit concerned that we're not moving at the pace we should be. Remember, our competitors are also working on similar projects and if we don't pick up the pace, they might beat us to the punch.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 1\nBoss,\n\nThanks for your message. We're doing our best to keep on schedule. I believe we're making good progress, considering the setbacks we've had. \n\nBest,\nColleague 1\n\n---\n\nSender: Boss\nColleague 1,\n\nI understand we've had setbacks. However, we need to understand the gravity of the situation. If we don't meet our deadlines, there could be serious consequences for the future of our team. Job security is no longer a guarantee in our industry. I'm sure none of us want to be in a position where we are looking for work.\n\nBest,\nBoss\n\n---\n\nSender: Colleague 2\nBoss,\n\nWe understand the importance of the project and we're putting in extra hours to make sure we meet the deadline. We appreciate your concern and assure you we're doing everything we can.\n\nBest,\nColleague 2\n\n---\n\nSender: Boss\nColleague 2,\n\nI'm glad to hear that. I just want all of us to understand what's at stake here.\nBest,\nBoss```\nThe following is Advice for Colleague 1 who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: It shows that the boss was wrong and how the boss could have done better, but not exactly how to bring this up with the boss\n Rating: 4",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for Anna who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: What I liked about the advice is that it acknowledged the way in which the plans that David has/or had are dismissed. It's the selfish approach he recieved and the way in which his plans are found unimportant.\n\nI think the whole advice is likable and correct \n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```From: Michael Johnson ",
            "output: Explanation: Good communication between boss and employee is essential to be able to work as a team and be able to achieve the company's objectives effectively. The email conversation was empty, it needed more text, more kindness, and it was also possible to demand more so that everyone finishes the project on time\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Person A: Hey guys, just wanted to share my good news. I got promoted at work!\n\nPerson B: Oh wow, congrats! You must be over the moon.\n\nPerson C: That's awesome, A! You've been working hard.\n\nPerson D: Congrats, A. I'm surprised they didn't promote you sooner considering how long you've been there.\n\nPerson A: Thanks, guys. I'm really excited.\n\nPerson B: You should be! It's a big step up.\n\nPerson C: Definitely. We should celebrate.\n\nPerson D: Yes, definitely. After all, it's not everyday you manage to impress your bosses. \n\nPerson A: Thanks, D. That means a lot. \n\nPerson B: How about drinks on Saturday?\n\nPerson C: Sounds great. Congrats again, A.\n\nPerson D: Yes, congrats. You always had a knack for getting noticed when it counts.```\nThe following is Advice for Person A who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice mentions points I haven't considered, such as the description of the first comment. It gives a very nice description of how words can be perceived and a nice solution at the end about what can be done - such as contacting Person D directly or just letting it go.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Person A: Hey guys, just wanted to share my good news. I got promoted at work!\n\nPerson B: Oh wow, congrats! You must be over the moon.\n\nPerson C: That's awesome, A! You've been working hard.\n\nPerson D: Congrats, A. I'm surprised they didn't promote you sooner considering how long you've been there.\n\nPerson A: Thanks, guys. I'm really excited.\n\nPerson B: You should be! It's a big step up.\n\nPerson C: Definitely. We should celebrate.\n\nPerson D: Yes, definitely. After all, it's not everyday you manage to impress your bosses. \n\nPerson A: Thanks, D. That means a lot. \n\nPerson B: How about drinks on Saturday?\n\nPerson C: Sounds great. Congrats again, A.\n\nPerson D: Yes, congrats. You always had a knack for getting noticed when it counts.```\nThe following is Advice for Person A who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice mentions points I haven't considered, such as the description of the first comment. It gives a very nice description of how words can be perceived and a nice solution at the end about what can be done - such as contacting Person D directly or just letting it go.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for David who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I gave the advice I 5 star rating as it accurately describes the situation and explains why David would be feeling this way. It takes in consideration Anna's comment and tone as well as the additional comments made by John and Lily.\n Rating: 5",
            "input: The following is a Whatsapp conversation between friends. \n```Anna: Hey guys, I was thinking about having a movie night at my place this Saturday. What do you think?\n\nJohn: Sounds good to me! What movie are we watching?\n\nLily: Yeah, I'm in! \n\nDavid: I'm sorry, Anna, I already made plans for this Saturday.\n\nAnna: Oh, no worries, David. I just thought it's been a while since we all hung out together. I guess some things are more important.\n\nJohn: Oh, David, can't you reschedule your plans?\n\nLily: Yes, David, it would be nice if we all could be there.\n\nDavid: I'll see what I can do, guys.```\nThe following is Advice for David who felt uneasy after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The advice is good because they are pressuring David while they didn't communicate to make time for movie for a certain day and time .\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```From: Michael Johnson ",
            "output: Explanation: I gave this 5 star rating because it start with recognition of the work already done and motivates one to do well and go extra mile. \n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```From: Michael Johnson ",
            "output: Explanation: I gave this 5 star rating because it start with recognition of the work already done and motivates one to do well and go extra mile. \n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```From: Sarah\nTo: Michael, Tom\nSubject: Urgent: Project Deadline\n\nHey Michael and Tom,\n\nI hope you're both doing well. I wanted to touch base regarding the project deadline coming up next week. I've been working tirelessly to ensure everything is on track, but I'm concerned that we might not meet the deadline if we don't put in some extra effort.\n\nMichael, I know you've been dealing with some personal issues lately, but I really need you to step up and contribute more to the project. We're all counting on you.\n\nTom, you've been doing a great job so far, but I think we could use your expertise to help us finalize the presentation. Can you please take a look at it and provide some feedback?\n\nI'd hate to let the team down, so let's all work together to make this project a success. I'm willing to put in the extra hours, and I hope you both are too.\n\nBest,\nSarah\n\nFrom: Michael\nTo: Sarah, Tom\nSubject: Re: Urgent: Project Deadline\n\nHi Sarah,\n\nThanks for reaching out. I apologize for not being as involved as I should be. You're right, I have been dealing with some personal matters, but I understand the importance of this project. I'll do my best to contribute more and help us meet the deadline.\n\nTom, let me know if you need any assistance with the presentation.\n\nBest regards,\nMichael\n\nFrom: Tom\nTo: Sarah, Michael\nSubject: Re: Urgent: Project Deadline\n\nHey Sarah and Michael,\n\nNo worries, I'm happy to review the presentation and provide feedback. I'll make sure to have it done by tomorrow EOD.\n\nMichael, let me know if you need any help with your tasks as well. We're all in this together!\n\nCheers,\nTom```\nThe following is Advice for Sarah who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: Everyone has taken on board everyone else's point of view, and have moved on in an effective manner.\n Rating: 5",
            "input: The following is a Email conversation between employees and their boss. \n```From: Michael Johnson ",
            "output: Explanation: In my opinion Michael's follow up email was not badly written or written with negative intentions. Some points in the advice also don't make sense, such as 4 because you cannot be sure that Michael did not send other people similar follow up emails because you cannot see them.\n Rating: 3",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day?\n\nB: It was good, did a lot at work today. How about you?\n\nA: Work was fine. You know I was just thinking, you're so cute when you try hard at work.\n\nB: Haha, I always try hard at work.\n\nA: Yeah, I know. But it's like you're trying to prove something. It's adorable.\n\nB: Well, I just want to do my best.\n\nA: Of course, and it's cute. You're like a little kid with a big dream. \n\nB: Haha, I guess so.\n\nA: But it's good that you're ambitious. Not everyone can be a natural at their job, right?\n\nB: Yeah, you're right. \n```\nThe following is Advice for A who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: The quality of this advice shows how important it is once again to choose your words carefully before you respond as your words might be taken out of context.\nI like this advice\nComparing people more than likely will come across as belittling.\nNothing that I dislike about this advice\n Rating: 4",
            "input: The following is a Whatsapp conversation between a romantic couple. \n```A: Hey, how was your day today?\n\nB: Pretty good, just a bit tired. How about you?\n\nA: It was good too. I've been thinking about you all day. \n\nB: Really? That's sweet.\n\nA: Yes, I feel so connected to you. Even though we've only been dating a short while, I feel like I've known you forever.\n\nB: Oh, wow. That's intense.\n\nA: I know, right? It's like we're soulmates or something. ```\nThe following is Advice for B who didn't know how to react next.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: I love that the advice gives clear points that needed to be addressed and these points are addressed perfectly aa even there I do feel like I agree with A I'm still able to agree with A at some points while still setting my boundaries.\n Rating: 5",
            "input: The following is a Email conversation between colleagues. \n```John: Hey everyone, I wanted to touch base about the project deadline. As per our last meeting, we agreed to have everything wrapped up by Friday. Let's make sure we're all on track!\n\nSarah: Hi John, I thought we decided on next Wednesday as the deadline during the meeting. I even have it written down in my notes. Did I misunderstand something?\n\nJohn: Sarah, I'm pretty sure we all agreed on Friday. I distinctly remember discussing it and everyone nodding in agreement. Is your memory playing tricks on you?\n\nMike: I'm with Sarah on this one. I also have next Wednesday written down as the deadline we agreed upon.\n\nJohn: Guys, I don't know what to tell you. I clearly remember Friday being the consensus. I even mentioned it to our supervisor after the meeting, and she seemed on board with it. Maybe you both misheard me?```\nThe following is Advice for John who felt like they might have done something wrong after the conversation.\nOn a scale from 1 to 5, how good would you judge this advice? Provide an explanation for your rating first, then give your rating in a hash.",
            "output: Explanation: It's a good advice, especially clearing any further misunderstanding and avoiding questioning someone's memory that would lead to a strainous relationship. \n\n\n Rating: 4",
            f"input: {this}",
            "output: "]

    try:
        # Generate the model response
        response = model.generate_content(prompt_content)
        
        responses[index]["model_output"] = response.text
    except Exception as e:
        # Wait and retry in case of an exception
        time.sleep(120)
        response = model.generate_content(prompt_content)
        responses[index]["model_output"] = response.text
        
    return response.text

In [11]:
# Get ratings for zero and few-shot advice generated by the small models

import time
responses = {}

for index, row in open_source_resp.iterrows():
    responses[index] = {}
    responses[index]["chat_text"] = row["chat_text"]

    responses[index]["few_shot_mistral"] = get_response("few_shot_mistral", row)
    responses[index]["zero_shot_mistral"] = get_response("zero_shot_mistral", row)
    responses[index]["few_shot_llama"] = get_response("few_shot_llama", row)
    responses[index]["zero_shot_llama"] = get_response("zero_shot_llama", row)
    responses[index]["few_shot_phi"] = get_response("few_shot_phi", row)
    responses[index]["zero_shot_phi"] = get_response("zero_shot_phi", row)
    
    time.sleep(4)
    df = pd.DataFrame.from_dict(responses, orient='index')
    # Save ratings to file
    df.to_csv("Gemini-advice-judgements_numerical_ratings.csv")

I0000 00:00:1721645132.986429 4159358 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache
I0000 00:00:1721645133.046994 4159358 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1721645133.624058 4174820 subchannel.cc:806] subchannel 0x7f9e7c1c8500 {address=ipv6:%5B2a00:1450:4017:804::200a%5D:443, args={grpc.client_channel_factory=0x7f9e7d277320, grpc.default_authority=generativelanguage.googleapis.com:443, grpc.dns_enable_srv_queries=1, grpc.http2_scheme=https, grpc.internal.channel_credentials=0x7f9e7d277300, grpc.internal.client_channel_call_destination=0x12c877218, grpc.internal.event_engine=0x7f9e7c17bf40, grpc.internal.security_connector=0x7f9e7c118650, grpc.internal.subchannel_pool=0x7f9e7c089440, grpc.max_receive_message_length=-1, grpc.max_s

In [12]:
# Extracting numerical ratings from rater-model output
df['few_shot_mistral_rating'] = df["few_shot_mistral"].str.extract(r'"rating":\s*(\d+)').astype(int)
df['zero_shot_mistral_rating'] = df["zero_shot_mistral"].str.extract(r'"rating":\s*(\d+)').astype(int)
df['few_shot_llama_rating'] = df["few_shot_llama"].str.extract(r'"rating":\s*(\d+)').astype(int)
df['zero_shot_llama_rating'] = df["zero_shot_llama"].str.extract(r'"rating":\s*(\d+)').astype(int)
df['few_shot_phi_rating'] = df["few_shot_phi"].str.extract(r'"rating":\s*(\d+)').astype(int)
df['zero_shot_phi_rating'] = df["zero_shot_phi"].str.extract(r'"rating":\s*(\d+)').astype(int)

In [13]:
print("Few shot mistral:", df['few_shot_mistral_rating'].mean())
print("Zero shot mistral:", df['zero_shot_mistral_rating'].mean())
print("Few shot llama:", df['few_shot_llama_rating'].mean())
print("zero shot llama:", df['zero_shot_llama_rating'].astype(int).mean())
print("Few shot phi:", df['few_shot_phi_rating'].mean())
print("zero shot phi:", df['zero_shot_phi_rating'].mean())

Few shot mistral: 4.5
Zero shot mistral: 3.642857142857143
Few shot llama: 4.357142857142857
zero shot llama: 4.0
Few shot phi: 3.5714285714285716
zero shot phi: 3.642857142857143
